In [ ]:
%%R
# Instalación de paquetes si no están instalados
if (!require('sf')) install.packages('sf')
if (!require('spdep')) install.packages('spdep')
if (!require('ggplot2')) install.packages('ggplot2')
if (!require('dplyr')) install.packages('dplyr')
if (!require('spatialreg')) install.packages('spatialreg')

# Cargar las librerías
library(sf)
library(spdep)
library(ggplot2)
library(dplyr)
library(spatialreg)


In [ ]:
%%R
# Descargar y leer los datos
temp <- tempfile()
download.file("https://github.com/rpasquini/urban-econometrics/blob/master/data/departamentos2019amba.zip?raw=true", temp)
unzip(temp, exdir = tempdir())

# Leer el shapefile con sf
departamentos <- st_read(file.path(tempdir(), "departamentos2019amba.shp"))

# Filtrar para CABA
departamentosCABA <- departamentos[departamentos$l2 == 'Capital Federal',]


In [ ]:
%%R
# Visualizar los datos con ggplot2
ggplot(data = departamentosCABA) +
  geom_sf() +
  theme_minimal() +
  labs(title = "Departamentos en CABA")


In [ ]:
%%R
# Tomar una submuestra y limpiar datos
set.seed(123)  # Para reproducibilidad
departamentosCABA2 <- departamentosCABA[1:500,]
departamentosCABA2 <- departamentosCABA2[departamentosCABA2$price < 80000,]
departamentosCABA2 <- departamentosCABA2[!is.na(departamentosCABA2$price) & !is.na(departamentosCABA2$bedrooms),]

# Visualizar precios
ggplot(data = departamentosCABA2) +
  geom_sf(aes(color = price)) +
  scale_color_viridis_c() +
  theme_minimal() +
  labs(title = "Precios de departamentos en CABA",
       color = "Precio")


In [ ]:
%%R
# Examinar el histograma de precios
ggplot(data = as.data.frame(departamentosCABA2), aes(x = price)) +
  geom_histogram(bins = 30, fill = "steelblue", color = "white") +
  theme_minimal() +
  labs(title = "Distribución de precios",
       x = "Precio",
       y = "Frecuencia")


In [ ]:
%%R
# Crear matriz de pesos basada en distancia
coords <- st_coordinates(st_centroid(departamentosCABA2))
dists <- dnearneigh(coords, 0, 1000)  # threshold de 1000 metros
wdistance <- nb2listw(dists, style="W")

# Visualizar la conectividad
plot(st_geometry(departamentosCABA2), main="Matriz de pesos espaciales")
plot(wdistance, coords, add=TRUE, col='red', lwd=0.5)


In [ ]:
%%R
# Calcular el I de Moran
moran_test <- moran.test(departamentosCABA2$price, wdistance)
print(moran_test)

# Gráfico de Moran
moran.plot(departamentosCABA2$price, wdistance,
           labels=FALSE, pch=16,
           xlab="Precio", ylab="Rezago espacial del precio",
           main="Diagrama de dispersión de Moran")


In [ ]:
%%R
# Crear matriz de pesos con k vecinos más cercanos
knn <- knearneigh(coords, k=10)
kernel_nb <- knn2nb(knn)
kernel_weights <- nb2listw(kernel_nb, style="W", zero.policy=TRUE)

# Visualizar
plot(st_geometry(departamentosCABA2), main="Matriz de pesos con k vecinos más cercanos")
plot(kernel_weights, coords, add=TRUE, col='blue', lwd=0.5)

# Calcular Moran con nuevos pesos
moran_kernel <- moran.test(departamentosCABA2$price, kernel_weights)
print(moran_kernel)


In [ ]:
%%R
# Regresión OLS
ols_model <- lm(price ~ bedrooms, data = departamentosCABA2)
summary(ols_model)

# Test de Moran en los residuos
moran.test(residuals(ols_model), wdistance)


In [ ]:
%%R
# Modelo de rezago espacial
spatial_lag <- lagsarlm(price ~ bedrooms, data = departamentosCABA2, listw = wdistance)
summary(spatial_lag)

# Impactos del modelo espacial
impacts(spatial_lag, listw = wdistance)
